# モジュールインポート

In [4]:
%load_ext autoreload
import pandas as pd
from tqdm.auto import tqdm
from graphviz import *
import lightgbm as lgb
from my_modules import prepare
import requests
from bs4 import BeautifulSoup
import re
import time


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# ディレクトリ構成

# ワイの環境

In [8]:
!python --version

Python 3.10.12


In [7]:
!python -m pip list

Package                 Version
----------------------- ---------
absl-py                 1.4.0
alembic                 1.11.1
appnope                 0.1.2
asttokens               2.0.5
astunparse              1.6.3
async-generator         1.10
attrs                   23.1.0
backcall                0.2.0
beautifulsoup4          4.12.2
boltons                 23.0.0
brotlipy                0.7.0
cachetools              5.3.1
certifi                 2023.7.22
cffi                    1.15.1
charset-normalizer      2.0.4
cloudpickle             2.2.1
cmaes                   0.10.0
colorlog                6.7.0
conda                   23.7.2
conda-content-trust     0.1.3
conda-package-handling  2.2.0
conda_package_streaming 0.9.0
contourpy               1.1.0
cryptography            39.0.1
cycler                  0.11.0
debugpy                 1.6.7
decorator               5.1.1
dill                    0.3.6
et-xmlfile              1.1.0
exceptiongroup          1.1.1
executing             

# PC 環境

<li>OS : Mac OS Sonoma 14.2.1</li>
<li>Chip : Apple M2 Pro</li>

# テスト用 race_id_list

In [11]:
kaisai_date_list = prepare.scrape_kaisai_date(from_="2023-01-01",to_='2023-02-01')
race_id_list = prepare.scrape_race_id_list(kaisai_date_list)

getting race date from 2023-01-01 to 2023-02-01


100%|██████████| 1/1 [00:01<00:00,  1.29s/it]


getting race_id_list


  0%|          | 0/10 [00:00<?, ?it/s]

scraping: https://race.netkeiba.com/top/race_list.html?kaisai_date=20230105


...

 80%|████████  | 8/10 [00:18<00:02,  1.28s/it]

scraping: https://race.netkeiba.com/top/race_list.html?kaisai_date=20230128


 90%|█████████ | 9/10 [00:19<00:01,  1.12s/it]

scraping: https://race.netkeiba.com/top/race_list.html?kaisai_date=20230129


100%|██████████| 10/10 [00:23<00:00,  2.40s/it]


# デバグ対象関数

In [9]:
SLEEP_TIME = 5 # サーバに負担をかけないよう, SLEEP _TIME だけ待つ

def scrape_race_results(race_id_list:list):
        #race_idをkeyにしてDataFrame型を格納
        # race_id : int
        race_results = {}
        for race_id in tqdm(race_id_list):
            time.sleep(SLEEP_TIME)
            print("race_id",race_id)
            try:
                url = "https://db.netkeiba.com/race/" + str(race_id)
                #メインとなるテーブルデータを取得
                df = pd.read_html(url)[0]
                html = requests.get(url)
                html.encoding = "EUC-JP"
                soup = BeautifulSoup(html.text, "html.parser")

                #天候、レースの種類、コースの長さ、馬場の状態、日付をスクレイピング
                texts = (
                    soup.find("div", attrs={"class": "data_intro"}).find_all("p")[0].text
                    + soup.find("div", attrs={"class": "data_intro"}).find_all("p")[1].text
                )
                info = re.findall(r'\w+', texts)
                print("info",info)
                print("texts",texts)
                for text in info:
                    if text in ["芝", "ダート"]:
                        df["race_type"] = [text] * len(df)
                    if "障" in text:
                        df["race_type"] = ["障害"] * len(df)
                    if "m" in text:
                        df["course_len"] = [int(re.findall(r"\d+", text)[0])] * len(df)
                    if text in ["良", "稍重", "重", "不良"]:
                        df["ground_state"] = [text] * len(df)
                    if text in ["曇", "晴", "雨", "小雨", "小雪", "雪"]:
                        df["weather"] = [text] * len(df)
                    if "年" in text:
                        df["date"] = [text] * len(df)

                #馬ID、騎手ID, owner_id, trainer_idをスクレイピング
                horse_id_list = []
                horse_a_list = soup.find("table", attrs={"summary": "レース結果"}).find_all(
                    "a",
                    attrs={"href": re.compile("^/horse")}
                )
                for a in horse_a_list:
                    horse_id = re.findall(r"\d+", a["href"])
                    horse_id_list.append(horse_id[0])
                jockey_id_list = []
                jockey_a_list = soup.find("table", attrs={"summary": "レース結果"}).find_all(
                    "a", attrs={"href": re.compile("^/jockey")}
                )
                for a in jockey_a_list:
                    jockey_id = re.findall(r"\d+", a["href"])
                    jockey_id_list.append(jockey_id[0])

                ############
                trainer_id_list = []
                trainer_a_list = soup.find("table", attrs={"summary": "レース結果"}).find_all(
                    "a", attrs={"href": re.compile("^/trainer")}
                )
                for a in trainer_a_list:
                    trainer_id = re.findall(r"\d+", a["href"])
                    trainer_id_list.append(trainer_id[0])
                
                ###########
                owner_id_list = []
                owner_a_list = soup.find("table", attrs={"summary": "レース結果"}).find_all(
                    "a", attrs={"href": re.compile("^/owner")}
                )
                for a in owner_a_list:
                    owner_id = re.findall(r"\d+", a["href"])
                    owner_id_list.append(owner_id[0])
                
                df["horse_id"] = horse_id_list
                df["jockey_id"] = jockey_id_list
                df["trainer_id"] = trainer_id_list
                df["owner_id"] = owner_id_list
                
                
                #インデックスをrace_idにする
                df.index = [race_id] * len(df)
                race_results[race_id] = df
            #存在しないrace_idを飛ばす
            except IndexError:
                print("IndexError")
                continue
                
            #wifiの接続が切れた時などでも途中までのデータを返せるようにする
            except Exception as e:
                print(e)
                break


        #pd.DataFrame型にして一つのデータにまとめる
        race_results_df = pd.concat([race_results[key] for key in race_results])
        return race_results_df

In [12]:
test_ = scrape_race_results(race_id_list[:2])

  0%|          | 0/2 [00:00<?, ?it/s]

race_id 202306010101


 50%|█████     | 1/2 [00:05<00:05,  5.87s/it]

info ['ダ右1200m', '天候', '晴', 'ダート', '良', '発走', '09', '55', '2023年1月5日', '1回中山1日目', '3歳未勝利', '馬齢']
texts 

ダ右1200m / 天候 : 晴 / ダート : 良 / 発走 : 09:55


2023年1月5日 1回中山1日目 3歳未勝利   (馬齢)
df               着 順  枠 番  馬 番         馬名  性齢  斤量    騎手     タイム     着差     単勝  \
202306010101    1    7   14  シュバルツガイスト  牡3  56    武豊  1:12.6    NaN    1.8   
202306010101    2    3    5  ジェイケイファイン  牡3  56  内田博幸  1:12.9  1.3/4    7.4   
202306010101    3    1    1      レディアス  牝3  54  津村明秀  1:12.9     クビ   36.1   
202306010101    4    8   15  ハローマイラブリー  牝3  54  三浦皇成  1:13.4      3    4.2   
202306010101    5    5    9  ベアカワイコチャン  牝3  54  松岡正海  1:13.9      3    7.9   
202306010101    6    1    2     ラディーヴァ  牝3  52   原優介  1:14.0    1/2  255.6   
202306010101    7    6   12  メルティーショコラ  牝3  54   武藤雅  1:14.0     クビ   58.8   
202306010101    8    8   16    ニシノスイセイ  牡3  56  野中悠太  1:14.1    1/2  137.1   
202306010101    9    7   13   タッカーカムカム  牝3  52  山田敬士  1:14.2     クビ  322.0   
202306010101   10    3    6    ペイシャコリン 

100%|██████████| 2/2 [00:11<00:00,  5.88s/it]

info ['ダ右1800m', '天候', '晴', 'ダート', '良', '発走', '10', '25', '2023年1月5日', '1回中山1日目', '3歳未勝利', '混', '馬齢']
texts 

ダ右1800m / 天候 : 晴 / ダート : 良 / 発走 : 10:25


2023年1月5日 1回中山1日目 3歳未勝利  (混)(馬齢)
df               着 順  枠 番  馬 番         馬名  性齢  斤量    騎手     タイム     着差     単勝  \
202306010102    1    4    8   プレミアペガサス  牡3  56  バシュロ  1:55.3    NaN   13.8   
202306010102    2    4    7   ジェントルタッチ  牡3  56   黛弘人  1:55.5  1.1/2   13.3   
202306010102    3    7   13  ニシノファンフェア  牡3  56  内田博幸  1:55.7      1   16.3   
202306010102    4    1    2  ピックアップライン  牡3  56  武士沢友  1:56.0      2   58.8   
202306010102    5    2    3    コレクテイニア  牡3  56   吉田豊  1:56.4      2    5.0   
202306010102    6    8   15    コスモメイゲツ  牡3  56  丹内祐次  1:56.4     ハナ   53.6   
202306010102    7    5   10   パーティーキング  牡3  56  木幡巧也  1:56.5    1/2   51.6   
202306010102    8    6   11  アフタヌーンドレス  牝3  54  田辺裕信  1:56.8      2    3.2   
202306010102    9    8   16     バンクサイド  牡3  56  木幡初也  1:56.9    1/2  226.2   
202306010102   10    3    6  エイチ